# Data Visualization Notebook

## Objectives

## Inputs

Images are taken from the test, train, validation folders and their subfolders.

* inputs/cherryleaves_dataset/leaf_images/train
* inputs/cherryleaves_dataset/leaf_images/validation
* inputs/cherryleaves_dataset/leaf_images/test

## Outputs

* Compute average image size from the train set. The CNN will train on the test set, hence is fundamental that all input images have the same size. Input size determines the CNN architecture.
* Image shape embeddings saved as .PKL.
* Plot mean and vairability of images per label.
* Plot to distinguish the contrast between healthy and mildew infected cherry leaves.
* Generate code that answers busines requirement 1 and can be used to build an image monatage on the Streamlit dashboard.

# Comments | Insights | Conclusions

These steps are necessary to furhter understand and prepare the data that will be fed to the CNN. Data is also visually arranged as requested by the client (Business Requirement #1)

# Import libraries

# Set working directory

We need to change the working directory from its current folder to its parent folder

* We access the current directory with os.getcwd()

Confirm the new current directory

## Set Input Directories

Set train, validation and test paths.

## Set output directory

## Set label names

# Data visualization of image data

## Compute average images size on train set

In computer vision, it is recommended to normalize image pixel values relative to the dataset mean and standard deviation. This helps to get consistent results when applying a model to new images and can also be useful for transfer learning. In practice, computing these statistics can be a little non-trivial since we usually can't load the whole dataset in memory and have to loop through it in batches.

## Images size for training the model is the average from all images in the train set.

## Save the image shape embeddings

# Average and variability of images per label

## Load images shape and labels in an array

## Plot and save mean and variability of images per label:

## Difference between average powdery-mildew infected and average healthy cherry leaves

## Image Montage

## Run Montage in all labels